In [1]:
import numpy as np
import pickle
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import os
from Cooking import checkAndCreateDir

from keras.models import Sequential
from keras.layers import Activation, Dropout, UpSampling2D, Conv2DTranspose, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, CSVLogger, EarlyStopping
from keras_tqdm import TQDMNotebookCallback

Using TensorFlow backend.


In [2]:
# Load training images
train_images = pickle.load(open("images/scene/out_reshaped_scenes.p", "rb" ))

# Load image labels
labels = pickle.load(open("images/labelled/out_1d_labels.p", "rb" ))

# Make into arrays as the neural network wants these
train_images = np.array(train_images)
labels = np.array(labels)

In [3]:
# Normalize labels - training images get normalized to start in the network
labels = labels / 255

# Shuffle images along with their labels, then split into training/validation sets
train_images, labels = shuffle(train_images, labels)
# Test size may be 10% or 20%
X_train, X_val, y_train, y_val = train_test_split(train_images, labels, test_size=0.1)

In [4]:
# Batch size, epochs and pool size below are all paramaters to fiddle with for optimization
batch_size = 128
epochs = 1000
pool_size = (2, 2)
input_shape = X_train.shape[1:]

input_shape

(80, 160, 3)

In [5]:
# Neural Network
model = Sequential()
# Normalizes incoming inputs. First layer needs the input shape to work
model.add(BatchNormalization(input_shape=input_shape))

# Below layers were re-named for easier reading of model summary; this not necessary
# Conv Layer 1
model.add(Conv2D(8, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv1'))

# Conv Layer 2
model.add(Conv2D(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv2'))

# Pooling 1
model.add(MaxPooling2D(pool_size=pool_size))

# Conv Layer 3
model.add(Conv2D(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv3'))
model.add(Dropout(0.2))

# Pooling 3
model.add(MaxPooling2D(pool_size=pool_size))

# Upsample 1
model.add(UpSampling2D(size=pool_size))

# Deconv 1
model.add(Conv2DTranspose(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv1'))
model.add(Dropout(0.2))

# Upsample 3
model.add(UpSampling2D(size=pool_size))

# Deconv 2
model.add(Conv2DTranspose(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv2'))

# Final layer - only including one channel so 1 filter
model.add(Conv2DTranspose(1, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Final'))

### End of network ###

In [6]:
# Using a generator to help the model use less data
# Channel shifts help with shadows slightly
datagen = ImageDataGenerator(channel_shift_range=0.2)
datagen.fit(X_train)

MODEL_OUTPUT_DIR = 'model/lanes'

plateau_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.0001, verbose=1)
checkpoint_filepath = os.path.join(MODEL_OUTPUT_DIR, 'models', '{0}_model.{1}-{2}.h5'.format('model', '{epoch:02d}', '{val_loss:.7f}'))
checkAndCreateDir(checkpoint_filepath)
checkpoint_callback = ModelCheckpoint(checkpoint_filepath, save_best_only=True, verbose=1)
csv_callback = CSVLogger(os.path.join(MODEL_OUTPUT_DIR, 'training_log.csv'))
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
callbacks=[plateau_callback, csv_callback, checkpoint_callback, early_stopping_callback]

In [7]:
# Compiling and training the model
model.compile(optimizer='Adam', loss='mean_squared_error')
model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size), steps_per_epoch=len(X_train)/batch_size, epochs=epochs, verbose=1, validation_data=(X_val, y_val), callbacks=callbacks)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/1000
6/5 [================================] - 29s 5s/step - loss: 0.0066 - val_loss: 0.0065
Epoch 2/1000
6/5 [================================] - 30s 5s/step - loss: 0.0065 - val_loss: 0.0064
Epoch 3/1000
6/5 [================================] - 26s 4s/step - loss: 0.0061 - val_loss: 0.0060
Epoch 4/1000
6/5 [================================] - 26s 4s/step - loss: 0.0047 - val_loss: 0.0041
Epoch 5/1000
6/5 [================================] - 28s 5s/step - loss: 0.0033 - val_loss: 0.0029
Epoch 6/1000
6/5 [================================] - 28s 5s/step - loss: 0.0024 - val_loss: 0.0028
Epoch 7/1000
6/5 [================================] - 31s 5s/step - loss: 0.0022 - val_loss: 0.0024
Epoch 8/1000
6/5 [================================] - 34s 6s/step - loss: 0.0020 - val_loss: 0.0019
Epoch 9/1000
6/5 [================================] - 32s 5s/step - loss: 0.0018 - val_loss: 0.0019
Epoch 10/1000
6/5 [========

6/5 [================================] - 26s 4s/step - loss: 9.2757e-04 - val_loss: 9.4680e-04
Epoch 46/1000
6/5 [================================] - 26s 4s/step - loss: 9.2514e-04 - val_loss: 9.7574e-04
Epoch 47/1000
6/5 [================================] - 30s 5s/step - loss: 9.1603e-04 - val_loss: 9.2106e-04
Epoch 48/1000
6/5 [================================] - 29s 5s/step - loss: 9.2781e-04 - val_loss: 9.7810e-04
Epoch 49/1000
6/5 [================================] - 31s 5s/step - loss: 9.2948e-04 - val_loss: 9.2952e-04
Epoch 50/1000
6/5 [================================] - 31s 5s/step - loss: 9.1167e-04 - val_loss: 9.3703e-04
Epoch 51/1000
6/5 [================================] - 30s 5s/step - loss: 9.0896e-04 - val_loss: 9.1923e-04
Epoch 52/1000
6/5 [================================] - 31s 5s/step - loss: 9.1202e-04 - val_loss: 9.3688e-04
Epoch 53/1000
6/5 [================================] - 30s 5s/step - loss: 9.1202e-04 - val_loss: 9.1396e-04
Epoch 54/1000
6/5 [==============

Epoch 91/1000
6/5 [================================] - 24s 4s/step - loss: 8.3992e-04 - val_loss: 8.6469e-04
Epoch 92/1000
6/5 [================================] - 24s 4s/step - loss: 8.5132e-04 - val_loss: 8.6590e-04
Epoch 93/1000
6/5 [================================] - 25s 4s/step - loss: 8.5082e-04 - val_loss: 8.4347e-04
Epoch 94/1000
6/5 [================================] - 24s 4s/step - loss: 8.4865e-04 - val_loss: 8.4167e-04
Epoch 95/1000
6/5 [================================] - 26s 4s/step - loss: 8.4028e-04 - val_loss: 8.3697e-04
Epoch 96/1000
6/5 [================================] - 26s 4s/step - loss: 8.3436e-04 - val_loss: 8.4314e-04
Epoch 97/1000
6/5 [================================] - 29s 5s/step - loss: 8.2699e-04 - val_loss: 8.3347e-04
Epoch 98/1000
6/5 [================================] - 27s 4s/step - loss: 8.3818e-04 - val_loss: 8.7581e-04
Epoch 99/1000
6/5 [================================] - 28s 5s/step - loss: 8.3126e-04 - val_loss: 8.3193e-04
Epoch 100/1000
6/5 

6/5 [================================] - 27s 4s/step - loss: 7.8220e-04 - val_loss: 7.9988e-04
Epoch 136/1000
6/5 [================================] - 28s 5s/step - loss: 7.7974e-04 - val_loss: 7.8560e-04
Epoch 137/1000
6/5 [================================] - 29s 5s/step - loss: 7.7824e-04 - val_loss: 7.8791e-04
Epoch 138/1000
6/5 [================================] - 30s 5s/step - loss: 7.9016e-04 - val_loss: 7.9161e-04
Epoch 139/1000
6/5 [================================] - 25s 4s/step - loss: 7.6923e-04 - val_loss: 7.8306e-04
Epoch 140/1000
6/5 [================================] - 27s 5s/step - loss: 7.7089e-04 - val_loss: 7.9471e-04
Epoch 141/1000
6/5 [================================] - 27s 5s/step - loss: 7.7395e-04 - val_loss: 7.8535e-04
Epoch 142/1000
6/5 [================================] - 26s 4s/step - loss: 7.7162e-04 - val_loss: 7.8335e-04
Epoch 143/1000
6/5 [================================] - 27s 4s/step - loss: 7.7217e-04 - val_loss: 7.9819e-04
Epoch 144/1000
6/5 [=====

Epoch 181/1000
6/5 [================================] - 25s 4s/step - loss: 7.3162e-04 - val_loss: 7.4445e-04
Epoch 182/1000
6/5 [================================] - 24s 4s/step - loss: 7.2146e-04 - val_loss: 7.4570e-04
Epoch 183/1000
6/5 [================================] - 25s 4s/step - loss: 7.1688e-04 - val_loss: 7.3986e-04
Epoch 184/1000
6/5 [================================] - 25s 4s/step - loss: 7.3062e-04 - val_loss: 7.4815e-04
Epoch 185/1000
6/5 [================================] - 28s 5s/step - loss: 7.3387e-04 - val_loss: 7.4313e-04
Epoch 186/1000
6/5 [================================] - 26s 4s/step - loss: 7.1774e-04 - val_loss: 7.3909e-04
Epoch 187/1000
6/5 [================================] - 28s 5s/step - loss: 7.2165e-04 - val_loss: 7.3563e-04
Epoch 188/1000
6/5 [================================] - 29s 5s/step - loss: 7.2267e-04 - val_loss: 7.3736e-04
Epoch 189/1000
6/5 [================================] - 29s 5s/step - loss: 7.3006e-04 - val_loss: 7.4559e-04
Epoch 190/

Epoch 204/1000
6/5 [================================] - 33s 5s/step - loss: 7.1557e-04 - val_loss: 7.2963e-04
Epoch 205/1000
6/5 [================================] - 32s 5s/step - loss: 7.0029e-04 - val_loss: 7.2183e-04
Epoch 206/1000
6/5 [================================] - 33s 5s/step - loss: 6.9769e-04 - val_loss: 7.2205e-04
Epoch 207/1000
6/5 [================================] - 33s 6s/step - loss: 7.1575e-04 - val_loss: 7.3143e-04
Epoch 208/1000
6/5 [================================] - 39s 6s/step - loss: 7.0021e-04 - val_loss: 7.2164e-04
Epoch 209/1000
6/5 [================================] - 38s 6s/step - loss: 6.9237e-04 - val_loss: 7.1651e-04
Epoch 210/1000
6/5 [================================] - 34s 6s/step - loss: 7.2275e-04 - val_loss: 7.3727e-04
Epoch 211/1000
6/5 [================================] - 40s 7s/step - loss: 7.0991e-04 - val_loss: 7.1782e-04
Epoch 212/1000
6/5 [================================] - 29s 5s/step - loss: 6.9737e-04 - val_loss: 7.1531e-04
Epoch 213/

In [8]:
# Freeze layers since training is done
model.trainable = False
model.compile(optimizer='Adam', loss='mean_squared_error')


# Save model architecture and weights
model.save('model/lanes/trained_lane_model_3_layer_new_2.h5')

# Show summary of model
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 80, 160, 3)        12        
_________________________________________________________________
Conv1 (Conv2D)               (None, 78, 158, 8)        224       
_________________________________________________________________
Conv2 (Conv2D)               (None, 76, 156, 16)       1168      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 38, 78, 16)        0         
_________________________________________________________________
Conv3 (Conv2D)               (None, 36, 76, 16)        2320      
_________________________________________________________________
dropout_1 (Dropout)          (None, 36, 76, 16)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 38, 16)        0         
__________